In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from google.colab import files # Dosya yükleme aracı
import warnings

warnings.filterwarnings('ignore')

# NLTK kaynaklarını indir (daha önce indirmediysen)
try:
    stopwords.words('english')
    WordNetLemmatizer()
except LookupError:
    nltk.download('stopwords')
    nltk.download('wordnet')
    print("NLTK kaynakları indirildi.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


NLTK kaynakları indirildi.


In [2]:
# Yükleme penceresini açar
print("Lütfen indirdiğiniz temizlenmiş 'clean_cv_data.csv' dosyasını seçin ve yükleyin.")
uploaded = files.upload()

# Yüklenen dosyanın adını al
file_name = list(uploaded.keys())[0]

# Dosyayı Pandas DataFrame olarak yükle
df = pd.read_csv(file_name)

print(f"\nDosya Başarıyla Yüklendi: {file_name}")
print(f"Toplam Satır Sayısı: {df.shape[0]}")
print("İlk 5 Satır:")
print(df.head())

Lütfen indirdiğiniz temizlenmiş 'clean_cv_data.csv' dosyasını seçin ve yükleyin.


Saving clean_cv_data.csv to clean_cv_data.csv

Dosya Başarıyla Yüklendi: clean_cv_data.csv
Toplam Satır Sayısı: 40001
İlk 5 Satır:
                                          clean_text main_label   sub_label
0                                 Jitesh Vishwakarma       meta      others
1             E-mail-Id: - jvishwakarma123@gmail.com       meta      others
2                       Contact Number: - 9960902548       meta      others
3                              PROFESSIONAL SUMMARY:     header  experience
4  · 4 years of technical experience in implement...    content  experience


In [6]:
# SÜTUN ADLARI GÜNCELLENDİ
TEXT_COLUMN = 'clean_text'  # Temizlenmiş metin sütununun yeni adı
LABEL_COLUMN = 'main_label' # Etiket sütununun yeni adı

# Bu veri setinde boş değerler, bir CV'nin parçası olan boş satırlardan kaynaklanabilir.
# Bu nedenle sadece etiket sütununda boş olanları atıyoruz.
# Ancak bu veri seti daha çok "CV metni" değil, "CV'nin küçük parçaları" gibi görünüyor.
# Bu durumda, her satırı tek bir "belge" gibi ele alacağız.
df.dropna(subset=[TEXT_COLUMN, LABEL_COLUMN], inplace=True)

# Eğitim ve Test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(
    df[TEXT_COLUMN],
    df[LABEL_COLUMN],
    test_size=0.20,      # %20 test verisi
    random_state=42,
    stratify=df[LABEL_COLUMN] # Sınıfların dengesini korumak için şart
)

print("\n--- Veri Bölme Tamamlandı ---")
print(f"Eğitim Seti (80%): {len(X_train)} CV/Satır")
print(f"Test Seti (20%): {len(X_test)} CV/Satır")


--- Veri Bölme Tamamlandı ---
Eğitim Seti (80%): 32000 CV/Satır
Test Seti (20%): 8000 CV/Satır


In [7]:
# Etiketleri (main_label) sayısal hale getir
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=3
)

print("\nTF-IDF Vektörleştirme Başlatıldı...")

X_train_vectors = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors = tfidf_vectorizer.transform(X_test)

print("--- TF-IDF Vektörleştirme Tamamlandı ---")
print(f"Sözlük Boyutu: {X_train_vectors.shape[1]}")


TF-IDF Vektörleştirme Başlatıldı...
--- TF-IDF Vektörleştirme Tamamlandı ---
Sözlük Boyutu: 10000


In [11]:
from sklearn.model_selection import GridSearchCV
# SVC'yi (Support Vector Classifier) kullanıyoruz ve çekirdeği 'linear' yapıyoruz
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 1. Sadece Linear SVM'in C parametresini optimize edelim:
# max_iter'ı çok daha yüksek tutmak zorundayız çünkü bu model yavaş yakınsayabilir.
svc_params = {
    'C': [0.1, 0.5, 1.0, 5.0, 10.0]
}

# 2. Grid Search'ü başlatma
# Linear SVM'i kullanıyoruz: SVC(kernel='linear')
svc_grid = GridSearchCV(
    SVC(kernel='linear', random_state=42, max_iter=10000, tol=1e-3),
    svc_params,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

# Eğitilmiş TF-IDF vektörleri üzerinde arama yap
print("Linear SVM (SVC(kernel='linear')) C parametresi optimize ediliyor...")
# Bu kısım LinearSVC'ye göre daha uzun sürebilir.
svc_grid.fit(X_train_vectors, y_train_encoded)

# Sonuçları göster
print("\n--- LINEAR SVM (SVC) OPTİMİZASYON SONUÇLARI ---")
print(f"En İyi Doğruluk (Cross-Validation): {svc_grid.best_score_:.4f}")
print(f"En İyi C Parametresi: {svc_grid.best_params_['C']}")

# 3. En iyi model ile son testi yap
best_svc_model = svc_grid.best_estimator_
y_pred_optimized = best_svc_model.predict(X_test_vectors)
accuracy_optimized = accuracy_score(y_test_encoded, y_pred_optimized)

print(f"\nTest Seti Üzerindeki Nihai Doğruluk (Linear SVM): {accuracy_optimized:.4f}")
target_names = label_encoder.classes_
print("Sınıflandırma Raporu:")
print(classification_report(y_test_encoded, y_pred_optimized, target_names=target_names, zero_division=0))

Linear SVM (SVC(kernel='linear')) C parametresi optimize ediliyor...
Fitting 5 folds for each of 5 candidates, totalling 25 fits

--- LINEAR SVM (SVC) OPTİMİZASYON SONUÇLARI ---
En İyi Doğruluk (Cross-Validation): 0.8721
En İyi C Parametresi: 1.0

Test Seti Üzerindeki Nihai Doğruluk (Linear SVM): 0.8744
Sınıflandırma Raporu:
              precision    recall  f1-score   support

     content       0.89      0.96      0.92      5468
      header       0.84      0.78      0.81      1070
        meta       0.83      0.64      0.72      1462

    accuracy                           0.87      8000
   macro avg       0.85      0.79      0.82      8000
weighted avg       0.87      0.87      0.87      8000

